# Model 1 (Random Forest Regression)

## Importing the necessary libraries

* `pandas`, `numpy`: data manipulation <br>
* `matplotlib`, `seaborn`: visualizations <br>
* `sklearn`: ML and deep learning frameworks <br>
* `OS`: to download and upload files


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
import os 

%load_ext autoreload
%autoreload 2


<h3>Getting data from kaggle<h3>

In [ ]:
DATASET_NAME = "sid321axn/beijing-multisite-airquality-data-set"
DOWNLOAD_DIR = os.path.join(os.getcwd(), 'data')


In [ ]:
from utilities.data_util import fetch_data

fetch_data(DATASET_NAME, DOWNLOAD_DIR)


### Combining all the dataset files into one file called `combined_data.csv`

In [ ]:
data_folder = os.path.join(os.getcwd(), 'data')
print(os.getcwd())

print("data_folder", data_folder)
all_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]

combined_data = pd.concat((pd.read_csv(file) for file in all_files), ignore_index=True)


print("shape" ,combined_data.shape)

combined_data.to_csv(os.path.join(data_folder, 'combined_data.csv'), index=False)

In [ ]:
data = pd.read_csv('data/combined_data.csv')
print("Shape:", data.shape)
data.head()

In [ ]:
data.info()

### There are missing values and random forest does not work if there are missing values, so we will find the missing values and fill them using forward_fill and backward_fill. 

In [ ]:
missing = data.isnull().sum()
print(missing[missing > 0])

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values')
plt.show()

In [ ]:
missing_percent = (data.isnull().sum() / len(data)) * 100
missing_values = (missing_percent[missing_percent > 0].sort_values(ascending=False))
print(missing_values)

In [ ]:
data = data.fillna(method='ffill')
data = data.fillna(method='bfill')

print("Values that are still missing", data.isnull().sum().sum())

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values after filling')
plt.show()

In [ ]:
data.head()

## Feature reduction
### Removing `No` and `station` as it does not affect AQI. <br>
### **Confirmed through trial and error**

In [ ]:
data_features = data.drop(columns=['No','station'])

In [ ]:
data_features.head()


### Dropping PM2.5 as that we will use our output label. Also removing PM10 as its closely linked to PM2.5 meaning its indicator of AQI, and we dont our model to learn from nearly identical variable that is closely linked to the output.

In [ ]:
X = data_features.drop(columns=['PM2.5','PM10'])
y = data_features['PM2.5']

print("X_shape", X.shape)
print("Y_shape", y.shape)

In [ ]:
X.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.hist(y, bins=30)
plt.title("PM2.5 Distribution")
plt.xlabel("Value")
plt.ylabel("Count")
plt.show()


### We will normalize the labels as models do better on normalized data

In [ ]:
y = np.log1p(y)  # Apply log transformation
plt.figure(figsize=(8, 6))
plt.hist(y, bins=30)
plt.title("Log of PM2.5 valuess")
plt.xlabel("Value")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.boxplot(y.dropna())  
plt.title("Box Plot of PM2.5")
plt.ylabel("PM2.5 Values ")
plt.show()

In [ ]:
import seaborn as sns

for column in X.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(X[column], kde=True, bins=30)
    plt.title(f"Distribution of {column}")
    plt.xlabel(column)
    plt.ylabel("Count")
    plt.show()

### Same here again we nomralizing the features as well

In [ ]:
features = ['RAIN', 'SO2', 'NO2', 'CO', 'O3']
for feature in features:
    X[feature] = np.log1p(X[feature])

In [ ]:
for column in X.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(X[column], kde=True, bins=30)
    plt.title(f"Distribution of {column}")
    plt.xlabel(column)
    plt.ylabel("Count")
    plt.show()

### We converting wind direction to one hot encoding, as random forest cant take strings, so we can convert the directions into integers. 

In [ ]:
X_encoded = pd.get_dummies(X, columns=['wd'], drop_first=True)

### We made the correlation matrix to see how correlated the features are, we dont have that many features and also they dont correlate that much, so we dont have to do feature reduction. 

In [ ]:

corr_matrix = X_encoded.corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
X_encoded.head()

### splitting the data based on years, into train and test

In [ ]:

train = X_encoded[(X_encoded['year'] >= 2013) & (X_encoded['year'] <= 2016)]
test = X_encoded[(X_encoded['year'] == 2017) & (X_encoded['month'] <= 2)]

X_train = train
y_train = y.loc[train.index]
X_test = test
y_test = y.loc[test.index]

### Normalizing the input as models train faster and with less bias on normalized input

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print( X_train_scaled[:5])

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

### Getting the model, and training, making predictions, and getting mean_absolute_error, mean_squared_error, r2_score
### for the hyperparamters, down in the code we used **Random Search CV** to get the best hyperparameters and reusing it here, as random search CV will take 20+ mins to run

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
rf_model = RandomForestRegressor(
    n_estimators=200,     
    min_samples_split=5,
    min_samples_leaf=1,
    max_features='log2',
    max_depth=None,   
    random_state=25,       
    n_jobs=-1,              
)

In [ ]:
rf_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

### Plotting the Acutual vs predicted

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values[:200], label='Actual')
plt.plot(y_pred[:200], label='Predicted')
plt.title("Actual vs Predicted (PM2.5)")
plt.ylabel("PM2.5 values")
plt.legend()
plt.show()

### Trying to figure out which features are most important

In [ ]:
importances = rf_model.feature_importances_
feature_names = X_train.columns

feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df)
plt.title("Random Forest Feature Importances")
plt.show()

### Removing those features that are not important

In [ ]:
low_importance_features = [
    'wd_NNW',
    'day',
    'wd_SW',
    'wd_WNW',
    'RAIN',
    'wd_SSW',
    'wd_ESE',
    'wd_SE',
    'wd_SSE',
    'wd_N',
    'wd_S',
    'wd_NNE',
    'wd_WSW',
    'wd_NE',
    'wd_ENE',
    'wd_W',
    'wd_NW',
]

X_encoded_reduced = X_encoded.drop(columns=low_importance_features)
train = X_encoded_reduced[(X_encoded_reduced['year'] >= 2013) & (X_encoded_reduced['year'] <= 2016)]
test = X_encoded_reduced[(X_encoded_reduced['year'] == 2017) & (X_encoded_reduced['month'] <= 2)]
X_train = train
y_train = y.loc[train.index]
X_test = test
y_test = y.loc[test.index]

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train.head()


In [ ]:
rf_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values[:200], label='Actual', alpha=0.7)
plt.plot(y_pred[:200], label='Predicted', alpha=0.7)
plt.title("Actual vs Predicted (PM2.5)")
plt.xlabel("Time (hours)")
plt.ylabel("PM2.5 values")
plt.legend()
plt.show()

In [ ]:
importances = rf_model.feature_importances_
feature_names = X_train.columns

feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df)
plt.title("Random Forest Feature Importances")
plt.show()

### Testing if we can remove more redundant features while keeping the accuracy at same level.

In [ ]:
train = train.drop(columns=['year', 'hour', 'month','TEMP','PRES','WSPM'])
test = test.drop(columns=['year', 'hour', 'month','TEMP','PRES','WSPM'])
X_train = train
y_train = y.loc[train.index]
X_test = test
y_test = y.loc[test.index]

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train.head()

In [ ]:
rf_model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values[:200], label='Actual', alpha=0.7)
plt.plot(y_pred[:200], label='Predicted', alpha=0.7)
plt.title("Actual vs Predicted (PM2.5)")
plt.xlabel("Time (hours)")
plt.ylabel("PM2.5 Values")
plt.legend()
plt.show()

In [ ]:
importances = rf_model.feature_importances_
feature_names = X_train.columns

feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df)
plt.title("Random Forest Feature Importances")
plt.show()

### Using Random Search CV to try to get the best hyperparameters. This part takes very long, so we have commented it, but it will take 30 mins approx

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}


In [ ]:
X_sample = X_encoded.sample(frac=0.1, random_state=42)
y_sample = y.loc[X_sample.index]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from joblib import parallel_backend

random_search = RandomizedSearchCV(
    rf_model, param_distributions=param_grid, n_iter=5, cv=3, random_state=42, n_jobs=-1
)

with parallel_backend('threading'):
    # Perform the random search
    random_search.fit(X_sample, y_sample)
print("Best Params", random_search.best_params_)

In [ ]:

with parallel_backend('threading'):
    random_search.fit(X_train_scaled, y_train)

In [ ]:
print("Best Params", random_search.best_params_)
print("Best RMSE", random_search.best_score_)


In [ ]:
best_rf = random_search.best_estimator_

y_pred_tuned = best_rf.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred_tuned)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
r2 = r2_score(y_test, y_pred_tuned)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

In [ ]:
y_pred = best_rf.predict(X_test_scaled) 
residuals = y_test - y_pred

plt.figure(figsize=(8,6))
plt.scatter(y_pred, residuals)
plt.axhline(y=0, linestyle='--')
plt.title("Residual Plot")
plt.xlabel("Predicted PM2.5 Value") 
plt.ylabel("Residual (Actual - Predicted)")
plt.show()

In [ ]:
importances = best_rf.feature_importances_
feat_imp_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df)
plt.title("Feature Importances of Random Search CV")
plt.show()

In [ ]:
importances = best_rf.feature_importances_
feature_names = X_train.columns if isinstance(X_train, pd.DataFrame) else range(X_train.shape[1])

feat_imp = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

for feature, imp in feat_imp[:10]:  # top 10
    print(feature, ":", imp)


### Graph for predicted vs Acutual AQI values

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title("Predicted vs. Actual AQI Values")
plt.xlabel("Actual AQI Value")
plt.ylabel("Predicted AQI Value")
plt.show()


## Conclusion
### We reached an accuracy of 89%. Some features such as day, temperature and pressure did not contriubute much to AQI according to feature importance, so we removed that as they dont affect the acuracy much as well. As shown by the above graph, it shows a good accuracy, and only does worse on low AQI values, in which it highly overpredicts the value compared to actual. One explanation we think is that there might be not that much data on low AQI values or maybe the sensor readings at low level have high noise. 